# Day 6: Lanternfish
https://adventofcode.com/2021/day/6

In [1]:
import numpy as np
import os
from typing import Optional
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
class LanternFish:
    """
    Each day we decrease the counter by 1.
    When the counter goes below 0, we
       - spawn a new fish with counter = 8
       - reset this fish to counter 6
    """

    counter:int 

    def __init__(self, init_counter:int = 8):
        self.counter = init_counter
    
    def day_older(self):  #  -> Optional[LanternFish]:
        baby: LanternFish = None
        if self.counter == 0:
            self.counter = 6
            baby = LanternFish()
        else:
            self.counter = self.counter - 1

        return baby

class School:
    """
    A collection of LanternFish
    """
    fish: list[LanternFish]

    def __init__(self, school_init: list[int]):
        self.fish = []
        for counter in school_init:
            new_fish = LanternFish(init_counter=counter)
            self.fish.append(new_fish)
    
    def day(self) -> None:
        babies: list[LanternFish] = []
        for fish in self.fish:
            baby = fish.day_older()
            if baby is not None:
                babies.append(baby)
        self.fish = self.fish + babies

    def count_fish(self) -> int:
        return len(self.fish)


In [3]:
# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_6'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_6'
) -> str:
    return os.path.join(data_directory, file_loc)

def read_input(input_file:str) -> list[int]:
    initial_state: list[int] = []
    with open(input_file) as f:
        for line in f:
            if line.rstrip():
                initial_state = initial_state + list([int(x) for x in line.split(",")])
    return initial_state


In [4]:
# Test using example data.

fish: list[int] = read_input(test_input_location())
school: School = School(school_init=fish)
for day in range(1, 81):
    school.day()
    if day == 18:
        assert school.count_fish() == 26
    if day == 80:
        assert school.count_fish() == 5934

In [5]:
# answer the question
fish: list[int] = read_input(input_location())
school: School = School(school_init=fish)
for day in range(1, 81):
    school.day()
school.count_fish()

352872

## Part 2 - 256 Days.

256 days is a lot of growth.  From 80 to 256, we see growth for the small sample data of 5934 to 26984457539.

This will mean a lot of memory and a lot of computing.

Attempt #1: Maybe we can project by estimatng some curve from sample data?

In [11]:
fish: list[int] = read_input(test_input_location())
school: School = School(school_init=fish)
y: list[int] = []
for day in range(1, 100):
    school.day()
    y.append(school.count_fish())

x_train = np.arange(1, 100).reshape(-1, 1)
Y_train = np.array(y)
ln_Y_train = np.log(Y_train)

regr = linear_model.LinearRegression()
regr.fit(x_train, ln_Y_train)

x_pred = np.arange(101, 257).reshape(-1, 1)
Y_pred = regr.predict(x_pred)

print('Slope: ', regr.coef_[0])
print('Intercept: ', regr.intercept_)

actual = 26984457539
prediction = np.exp(regr.coef_[0] * 256 + regr.intercept_)
print("predicted val = {:f}".format(prediction))
print(f"diff {prediction - actual} or {prediction/actual}")

Slope:  0.08722023723097547
Intercept:  1.707898564240363
predicted val = 27467773875.159622
diff 483316336.1596222 or 1.0179109154023607


Result of attempt 1.  No!  Off by less than 2%, but still off... 

Attempt 2.
Instead of creating indivdual fish, let's keep track of counts of fish of each class (days left).

In [7]:
def count_fish(fish: list[int], days:int) -> int:
    fish_counts = dict(Counter(fish))

    for day in range(days):
        reproducing = fish_counts.get(0, 0)
        
        fish_counts[0] = 0
        for i in range(8):
            fish_counts[i] = fish_counts.get(i + 1, 0)
        
        fish_counts[8] = reproducing
        fish_counts[6] = fish_counts.get(6, 0) + reproducing

    return sum([v for v in dict(fish_counts).values()])

fish: list[int] = read_input(test_input_location())
assert count_fish(fish, 18) == 26
assert count_fish(fish, 80) == 5934
assert count_fish(fish, 256) == 26984457539

In [8]:
fish: list[int] = read_input(input_location())
count_fish(fish, 256)

1604361182149

*whew ... that worked.*